## Question 3

### Importing relevant libraries

In [1]:
import io
from io import BytesIO
import requests
import pandas as pd
import numpy as np
import calendar
from datetime import timedelta, date
import datetime as datetime
from dateutil.relativedelta import relativedelta
from dateutil import rrule
import undetected_chromedriver as uc
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import warnings
warnings.filterwarnings('ignore')

C:\Users\MANAS\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\MANAS\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\MANAS\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


C:\Users\MANAS\anaconda3\lib\site-packages\pyfolio\pos.py:26: UserWarning:

Module "zipline.assets" not found; multipliers will not be applied to position notionals.



### Formatting data

In [2]:
def clear_entire_text(element): """ Sends specific number of backspaces as keys """ 
    element.send_keys(Keys.CONTROL + 'a', Keys.BACKSPACE)

In [3]:
def month_end(start,end):  """ Calculates the end of month date and formats it in the appropriate type """
    
    dateList = []
    
    while start<=end:
        
        res = calendar.monthrange(start.year, start.month)[1]
        if start.month <= 9:
            if res <= 9:
                index = "0"+str(res)+"-0"+str(start.month)+"-"+str(start.year)
            else:
                index = str(res)+"-0"+str(start.month)+"-"+str(start.year)
        else:
            index = str(res)+"-"+str(start.month)+"-"+str(start.year)
        
        dateList.append(index)
        start = start + timedelta(days = abs(start.day-res)+1)
        
    return dateList

### Arguments

In [7]:
# Data is fetched for a shorter period to save time. 
# However, we can simply pass the required time frame by modifying the start and end variables

dList = []
start = date(2010,2,5)
end = date(2010,4,30)
if end.month == date.today().month and end.year == date.today().year: 
    end = end - timedelta(days = end.day)
dList = month_end(start, end)

In [8]:
dList

['28-02-2010', '31-03-2010', '30-04-2010']

### Extracting Data

In [9]:
driver = uc.Chrome()
driver.get(f'https://www.sebi.gov.in/sebiweb/other/OtherAction.do?doMfdTrends=yes')
time.sleep(3)
l=driver.find_elements_by_xpath("//*[@id=\"member-wrapper\"]/section/div[3]/div/table/thead/tr/th")

cols= []

for i in l:
    cols.append(i.text)
    
df_eq = pd.DataFrame(columns = cols)
df_db = pd.DataFrame(columns = cols)

db = []
eq = []
gp_eq = []
gs_eq = []
ni_eq = []
db_eq = []
gp_db = []
gs_db = []
ni_db = []
datel = []

for date in dList:
    
    time.sleep(3)

    fromdate = WebDriverWait(driver, 10).until(lambda x: x.find_element_by_xpath('//*[@id="date"]'))
    clear_entire_text(fromdate)
    time.sleep(3)
    fromdate.send_keys(date)

    time.sleep(3)

    continue_btn = WebDriverWait(driver, 10).until(lambda x: x.find_element_by_xpath('//*[@id="member-wrapper"]/section/div[1]/div/div[3]/a'))
    continue_btn.click()

    time.sleep(5)
    
    table_trs = driver.find_elements(By.XPATH, '//*[@id="member-wrapper"]/section/div[3]/div/table/tbody/tr')

    for row, i in zip(table_trs[:-2],range(0, len(table_trs[:-3]))):

        if i % 2 == 0: 
            datel.append(row.find_elements(By.TAG_NAME, "td")[0].text)

            eq.append(row.find_elements(By.TAG_NAME, "td")[1].text)
            gp_eq.append(row.find_elements(By.TAG_NAME, "td")[2].text)
            gs_eq.append(row.find_elements(By.TAG_NAME, "td")[3].text)
            ni_eq.append(row.find_elements(By.TAG_NAME, "td")[4].text)        

        else:
            
            db.append(row.find_elements(By.TAG_NAME, "td")[0].text)
            gp_db.append(row.find_elements(By.TAG_NAME, "td")[1].text)
            gs_db.append(row.find_elements(By.TAG_NAME, "td")[2].text)
            ni_db.append(row.find_elements(By.TAG_NAME, "td")[3].text)  
    print(f"Extracted the data for the date : {date}")

Extracted the data for the date : 28-02-2010
Extracted the data for the date : 31-03-2010
Extracted the data for the date : 30-04-2010


In [ ]:
df_eq['Trading Date'] = pd.Series(datel)
df_eq['Debt/Equity'] = pd.Series(eq)
df_eq['Gross Purchases(Rs Crore)'] = pd.Series(gp_eq)
df_eq['Gross Sales(Rs Crore)'] = pd.Series(gs_eq)
df_eq['Net Investment (Rs Crore)'] = pd.Series(ni_eq)


df_db['Trading Date'] = pd.Series(datel)
df_db['Debt/Equity'] = pd.Series(db)
df_db['Gross Purchases(Rs Crore)'] = pd.Series(gp_db)
df_db['Gross Sales(Rs Crore)'] = pd.Series(gs_db)
df_db['Net Investment (Rs Crore)'] = pd.Series(ni_db)

### Output

In [ ]:
df_db

,Trading Date,Debt/Equity,Gross Purchases(Rs Crore),Gross Sales(Rs Crore),Net Investment (Rs Crore)
0,"01 Feb, 2010",Debt,1785.48,1723.95,61.53
1,"02 Feb, 2010",Debt,2319.86,1457.81,862.05
2,"03 Feb, 2010",Debt,1854.63,2362.03,(507.40)
3,"04 Feb, 2010",Debt,2733.38,2438.12,295.26
4,"05 Feb, 2010",Debt,3485.92,1571.07,1914.85
...,...,...,...,...,...
57,"25 Mar, 2010",Debt,5257.49,6646.24,(1388.75)
58,"26 Mar, 2010",Debt,3225.18,8073.77,(4848.59)
59,"29 Mar, 2010",Debt,4237.83,7177.79,(2939.96)
60,"30 Mar, 2010",Debt,3221.05,10459.06,(7238.01)


In [ ]:
df_eq

,Trading Date,Debt/Equity,Gross Purchases(Rs Crore),Gross Sales(Rs Crore),Net Investment (Rs Crore)
0,"01 Feb, 2010",Equity,595.05,562.54,32.51
1,"02 Feb, 2010",Equity,417.57,664.55,(246.98)
2,"03 Feb, 2010",Equity,752.25,746.10,6.15
3,"04 Feb, 2010",Equity,561.70,857.92,(296.22)
4,"05 Feb, 2010",Equity,672.00,678.75,(6.75)
...,...,...,...,...,...
57,"25 Mar, 2010",Equity,1083.57,1154.39,(70.82)
58,"26 Mar, 2010",Equity,578.74,780.53,(201.79)
59,"29 Mar, 2010",Equity,570.42,952.67,(382.25)
60,"30 Mar, 2010",Equity,465.84,566.04,(100.20)


### Saving to Excel

In [ ]:
df_eq.to_excel("Equity.xlsx")
df_db.to_excel("Debt.xlsx")